# Elasticsearch

In [ ]:
import os
from elasticsearch import Elasticsearch, helpers, NotFoundError
import json
from datetime import datetime

In [ ]:
USER = "elastic"
PWD = "mXwp5dz4"
API_KEY = "a1JVeVo1VUJBdUJsalpERXYwNXg6RnpOeEZvUnRTTC0xZVJDQ0ZacHhRdw=="

In [ ]:
client = 

In [ ]:
client.??

In [ ]:
client = 

In [ ]:
client.info()

In [ ]:
health = client.cluster.health()
health.items()

### Elasticsearch Python API

- documentation: https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html

### Indexing a document

In [ ]:
doc = {
    'author': 'Meenakshi Syamkumar',
    'text': 'Hope you have been enjoying CS639!',
    'timestamp': datetime.now(),
}


In [ ]:
doc = {
    'author': 'Meenakshi Syamkumar',
    'text': 'It is time we learn how to handle unstructured data!',
    'timestamp': datetime.now(),
}


### Getting a document

In [ ]:
resp = 
print(resp)

In [ ]:
print()

In [ ]:
resp = client.get(index="test-index", id=2)
print(resp)

### Searching for all matching documents

- documentation: https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-all-query.html

In [ ]:
resp = 
resp

### Deleting a document

In [ ]:
client.

In [ ]:
resp = client.
resp

### New York City Restaurants JSON dataset

In [ ]:
nyc_index = "nyc_restaurants"

In [ ]:
client.indices.create(index=nyc_index)

In [ ]:
json_data_dir = "data/json"
json_files = [f for f in os.listdir(json_data_dir) if ".json" in f]
json_files

### Bulk loading JSON using `helpers.bulk` function

```
operations = [
    {"index": "my_index", "_source": {"field1": "value1", "field2": "value2"}},
    {"index": "my_index", "_source": {"field1": "value3", "field2": "value4"}},
    ...
]
response = helpers.bulk(client, operations)
```
documentation: https://elasticsearch-py.readthedocs.io/en/latest/helpers.html

In [ ]:
try:
    client.indices.delete(index=nyc_index)
except NotFoundError as e:
    print("Index doesn't exist!")

In [ ]:
client.indices.create(index=nyc_index)

### `get_mappings` method

- Retrieves the mapping definition of an index or indices.
- The mapping defines how the documents and fields within the index are structured, specifying data types (like text, keyword, integer, etc.) and configurations like analyzers and index settings.

In [ ]:
client.
print(json.dumps(dict(mapping), indent=4))

### Bulk loading TXT using `helpers.bulk` function

In [ ]:
txt_data_dir = "data/text"
txt_files = [f for f in os.listdir(txt_data_dir) if ".txt" in f]
txt_files

In [ ]:
nyc_wiki_index = "nyc_restaurants_wiki"

In [ ]:
mapping = 
print(json.dumps(dict(mapping), indent=4))

#### Query to match all documents

In [ ]:
query = {
    
}

resp = 
resp

In [ ]:
query = {
    
}

resp = client.search(index=nyc_wiki_index, body=query)
resp

### Full Query DSL (Domain Specific Language)

### `search` method

- Allows you to execute a search query and get back search hits that match the query. You can provide search queries using the q query string parameter or request body.
- documentation: https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html

#### (Simple search) Q1: Find all restaurants in Manhattan that were subject to inspection.

In [ ]:
response = client.search(
    index=nyc_index,
    body=
)

for hit in response['hits']['hits']:
    print(hit['_source']['DBA'], hit['_source']['BORO'], \
         hit['_source']['BUILDING'], hit['_source']['STREET'])

### Controlling size of the `hits` and the `_source` fields

In [ ]:
response = client.search(
    index=nyc_index,
    body=
)

print(response['hits']['total']['value'])

# for hit in response['hits']['hits']:
#     # print(hit['_source']['DBA'], hit['_source']['BORO'], \
#     #      hit['_source']['BUILDING'], hit['_source']['STREET'])
#     print(hit['_source'])

#### (Simple search) Q2: Find all inspected "Pizza" restaurants.

In [ ]:
response = client.search(
    index=nyc_index,
    body=
)

for hit in response['hits']['hits']:
    print(hit['_source']['DBA'])

#### (Fuzzy search) Q3: Find all inspected restaurants whose names are similar to "Mamma Mia".

In [ ]:
response = client.search(
    index=nyc_index,
    body=
)

for hit in response['hits']['hits']:
    print(hit['_source']['DBA'])



#### (Phrase match search) Q4: Find all inspected restaurants whose violations that have the phrase "food worker".

In [ ]:
response = client.search(
    index=nyc_index,
    body=
)

for hit in response['hits']['hits']:
    print(hit['_source']['DBA'], \
         hit['_source']['VIOLATION DESCRIPTION'])

#### (Multi field match search) Q5: Find all restaurants that have "Pizza" or "Pasta" in either their name or their "CUISINE DESCRIPTION".

In [ ]:
response = client.search(
    index="nyc_restaurants",
    body=
)

for hit in response['hits']['hits']:
    print(f"{hit['_source']['DBA']}: {hit['_source']['CUISINE DESCRIPTION']}")

#### Q6: Find all inspected restaurants that have Italian or Mexican in their "CUISINE DESCRIPTION".

Using `multi_match`.

In [ ]:
query = {
    
}

response = client.search(index=nyc_index, body=query)
print(response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit['_source']['DBA'], hit['_source']['CUISINE DESCRIPTION'])

Using `simple_query_string`.

In [ ]:
query = {
    
}
response = client.search(index=nyc_index, body=query)
print(response['hits']['total']['value'])

for hit in response['hits']['hits']:
    print(hit['_source']['DBA'], ":", hit['_source']['CUISINE DESCRIPTION'])


### Boosting

- Boosting is the process by which you can modify the relevance of a document.
- There are two different types of boosting: boost at indexing or boost while querying.
- Reading: https://weng.gitbooks.io/elasticsearch-in-action/content/chapter6_searching_with_relevancy/63boosting.html 

#### Q7: Find all inspected restaurants that have Italian or Mexican in their "CUISINE DESCRIPTION" with higher scoring for Mexican (10.0).

In [ ]:
query = {
    
}
response = client.search(index=nyc_index, body=query)
print(response['hits']['total']['value'])

for hit in response['hits']['hits']:
    print(hit['_source']['DBA'], ":", hit['_source']['CUISINE DESCRIPTION'])


### Why does Italian still get ranked higher than Mexican?

1. Term Frequency: If the term "Italian" appears more frequently across documents, Elasticsearch’s default ranking algorithm (BM25) may still give it higher weight despite the boost.

2. Document Length: If documents mentioning "Mexican" are much longer, Elasticsearch may consider the match to be less significant, while shorter documents mentioning "Italian" may rank higher.

3. Field Analysis: The field `VIOLATION DESCRIPTION` might be tokenized in a way that affects how terms are matched and scored. For example, stemming or normalization could influence relevance.

4. Scoring Nuances: Elasticsearch's scoring formula (BM25) factors in other elements like field length, inverse document frequency (IDF), and term frequency (TF), which could make "Italian" rank higher if it fits better with these parameters in the indexed documents.

### How can we re-write the query to make sure `Mexican` gets boosted?

### Disjunction max query

- Returns documents matching one or more wrapped queries, called query clauses or clauses.
- If a returned document matches multiple query clauses, the dis_max query assigns the document the highest relevance score from any matching clause, plus a tie breaking increment for any additional matching subqueries.
- documentation: https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-dis-max-query.html 

In [ ]:
query = {
    
}

response = client.search(index=nyc_index, body=query)
print(response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit['_source']['DBA'], ":", hit['_source']['CUISINE DESCRIPTION'])


### Highlighting 

- During the search process, elasticsearch extracts the text from the fields you want to highlight.
- It then marks the matching terms in the retrieved documents, usually by wrapping them in HTML tags (like `<em>` or `<strong>`), making them visually distinct.

#### Q8: Find all inspected restaurants that have Mexican in "CUISINE DESCRIPTION" and highlight the "CUISINE DESCRIPTION" field in the results.

In [ ]:
query = {

}

response = client.search(index=nyc_index, body=query)
print(response)

In [ ]:
query = {
    "query": {
        "match_phrase": {
            "CUISINE DESCRIPTION": "Mexican"
        }
    },
    "_source": ["DBA", "CUISINE DESCRIPTION"],
    "highlight": {
        "fields": {
            "CUISINE DESCRIPTION": {
                "pre_tags": ["<strong>"], 
                "post_tags": ["</strong>"]
            }
        }
    }
}

response = client.search(index=nyc_index, body=query)
print(response)

### Benefits of Highlighting
- Improved User Experience: Highlighting helps users quickly identify relevant sections of text that match their search terms, making it easier to evaluate the results.
- Enhanced Readability: By drawing attention to specific keywords or phrases, you improve the overall readability of search results.
- Customization: You can customize how highlighting appears (e.g., using different tags or styles) to fit the design of your application.

### Boolean operators

- `must` operator: AND
- `should` operator: OR
- `must_not` operator: NOT

#### Q9: Find all Italian restaurants that were inspected in Manhattan.

In [ ]:
query = {
    
}

response = client.search(index=nyc_index, body=query)
print(response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit['_source'])

#### Q10: Find all inspected restaurants that have Italian or Mexican in their "CUISINE DESCRIPTION".

In [ ]:
query = {
    
}

response = client.search(index=nyc_index, body=query)
print(response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit['_source'])

#### Q11: Find all inspected restaurants that are not in Bronx.

In [ ]:
query = {
    
}

response = client.search(index=nyc_index, body=query)
print(response['hits']['total']['value'])
for hit in response['hits']['hits']:
    print(hit['_source'])